In [5]:
import pandas as pd
df=pd.read_csv("iyc.csv")
df1=pd.read_csv("iyv.csv")
df.info()
df1.info()
df.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   filename            200 non-null    object
 1   Category            200 non-null    int64 
 2   Background          200 non-null    int64 
 3   Shape               200 non-null    int64 
 4   Orientation         200 non-null    int64 
 5   Color               200 non-null    int64 
 6   Lighting            200 non-null    int64 
 7   Reflection          200 non-null    int64 
 8   Texture             200 non-null    int64 
 9   Optical Properties  200 non-null    int64 
 10  DOP                 200 non-null    int64 
dtypes: int64(10), object(1)
memory usage: 17.3+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 87 entries, 0 to 86
Data columns (total 11 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   filename       

,filename,Category,Background,Shape,Orientation,Color,Lighting,Reflection,Texture,Optical Properties,DOP
0,glass_cup_45 degrees ( Black).JPG,3,30,600,50,0,0,66,120,7,1
1,glass_cup_45 degrees ( elevated).JPG,3,20,600,50,0,0,66,120,7,1
2,glass_cup_45 degrees (2).JPG,3,20,600,50,0,0,66,120,7,1
3,glass_cup_45 degrees (black elevated) (2).JPG,3,20,600,50,0,0,66,120,7,1
4,glass_cup_45 degrees (black elevated) (3).JPG,3,20,600,50,0,0,66,120,7,1


In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

# Assuming input images are 64x64 pixels with 3 color channels (RGB)
input_shape = (64, 64, 3)

# Create a Sequential model
model = Sequential()

# First Convolutional Layer
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
model.add(MaxPooling2D((2, 2)))

# Second Convolutional Layer
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))

# Flatten the output for the fully connected layers
model.add(Flatten())

# Fully Connected Layer
model.add(Dense(128, activation='relu'))

# Output layer with softmax activation for multiclass classification
model.add(Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 31, 31, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 12544)             0         
                                                                 
 dense_2 (Dense)             (None, 128)              

In [53]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

# Separate features (X) and labels (y) for training set
X_train = df.drop(columns=['filename'])
y_train = df['filename']

# Separate features (X) for testing set
X_test = df1.drop(columns=['filename'])

# Use LabelEncoder for encoding categorical labels
label_encoder = LabelEncoder()

# Encode the categorical labels
y_train_encoded = label_encoder.fit_transform(y_train)

# Convert encoded labels to one-hot encoding
num_classes = len(label_encoder.classes_)
y_train_one_hot = to_categorical(y_train_encoded, num_classes=num_classes)

# Normalize numerical attributes (scaling) for training set
scaler = StandardScaler()
X_train_normalized = scaler.fit_transform(X_train)

# Normalize numerical attributes (scaling) for testing set
X_test_normalized = scaler.transform(X_test)

# Define the model architecture for multi-class classification
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(X_train_normalized.shape[1],)))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))  # Output layer with softmax for multi-class

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model on the training data
model.fit(X_train_normalized, y_train_one_hot, epochs=30, batch_size=16)

# Predict filenames for the test set
predictions = model.predict(X_test_normalized)

# Decode the predictions using inverse_transform
predicted_filenames_encoded = predictions.argmax(axis=1)
predicted_filenames = label_encoder.inverse_transform(predicted_filenames_encoded)
# Display the predicted filenames

def categorize_class(filename):
    if 'glass_cup' in filename.lower():
        return 'Glass Cup'
    elif 'glass_bottle' in filename.lower():
        return 'Glass Bottle'
    elif 'metal_spring' in filename.lower():
        return 'Metal Spring'
    elif 'plastic_bottle' in filename.lower():
        return 'Plastic Bottle'
    elif 'plastic_cup' in filename.lower():
        return 'Plastic Cup'
    elif 'metal_nut' in filename.lower():
        return 'Metal Nut'
    else:
        return 'Class Miscellaneous'

# Apply the function to categorize classes for each filename in the DataFrame
result_df['True Class'] = result_df['True Filename'].apply(lambda x: categorize_class(x))
result_df['Predicted Class'] = result_df['Predicted Filename'].apply(lambda x: categorize_class(x))
result_df = result_df.rename(columns={'_1': 'True Labels'})
# Display the DataFrame with the updated column name
accuracy = accuracy_score(result_df['True Class'], result_df['Predicted Class'])

# Display the DataFrame with the updated column name and accuracy
print(result_df)
print(f"\nAccuracy: {accuracy}")
classification_rep = classification_report(result_df['True Class'], result_df['Predicted Class'])
print("\nClassification Report:")
print(classification_rep)


Epoch 1/30
13/13 [==============================] - 1s 2ms/step - loss: 5.3026 - accuracy: 0.0000e+00
Epoch 2/30
13/13 [==============================] - 0s 2ms/step - loss: 5.2591 - accuracy: 0.0150
Epoch 3/30
13/13 [==============================] - 0s 2ms/step - loss: 5.2191 - accuracy: 0.0350
Epoch 4/30
13/13 [==============================] - 0s 3ms/step - loss: 5.1692 - accuracy: 0.0300
Epoch 5/30
13/13 [==============================] - 0s 3ms/step - loss: 5.1048 - accuracy: 0.0350
Epoch 6/30
13/13 [==============================] - 0s 3ms/step - loss: 5.0142 - accuracy: 0.0350
Epoch 7/30
13/13 [==============================] - 0s 2ms/step - loss: 4.8991 - accuracy: 0.0400
Epoch 8/30
13/13 [==============================] - 0s 4ms/step - loss: 4.7554 - accuracy: 0.0350
Epoch 9/30
13/13 [==============================] - 0s 2ms/step - loss: 4.5973 - accuracy: 0.0350
Epoch 10/30
13/13 [==============================] - 0s 2ms/step - loss: 4.4333 - accuracy: 0.0300
Epoch 11/30
13/